$\textbf{LIBRARIES}$

In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [76]:
def sep_print(): print('='*100)

In [77]:
path = "../Data/"

Quick look into data

In [78]:
df_states = pd.read_csv(path + 'stations_info.csv')
print(df_states.columns)

sep_print()
print(df_states.head(2))

Index(['file_name', 'state', 'city', 'agency', 'station_location',
       'start_month', 'start_month_num', 'start_year'],
      dtype='object')
  file_name           state        city agency          station_location  \
0     AP001  Andhra Pradesh    Tirupati  APPCB       Tirumala, Tirupati    
1     AP002  Andhra Pradesh  Vijayawada  APPCB  PWD Grounds, Vijayawada    

  start_month  start_month_num  start_year  
0        July                7        2016  
1         May                5        2017  


I have a lot of data, so we're going to choose only one state for analysis. 
I'm looking for the state that has the minimum number of agencies (each agency can have different measurements or columns name for the same feature, so choosing one will be easier) and the most data available and similar number of columns.

In [79]:
sum_dict = dict()
max_dict = dict()
min_dict = dict()

for i in os.listdir(path):

    if not i.endswith(".csv"):
        next

    prefix = i[:2]
    val = pd.read_csv(path + i, nrows= 1).shape[1]

    if prefix in sum_dict:
            sum_dict[prefix] += val
            max_dict[prefix] = val if max_dict[prefix] < val else  max_dict[prefix]
            min_dict[prefix] = val if min_dict[prefix] > val else  min_dict[prefix]
    else:
        sum_dict[prefix] = val
        max_dict[prefix] = val
        min_dict[prefix] = val

smm_df = pd.DataFrame({'sum': sum_dict.values(), 'min': min_dict.values(), 'max': max_dict.values()}, index=sum_dict.keys()).sort_values(by= ["sum", "min", "max"])


In [80]:
#Namber of diffrent agencies that measure air quility
df_states["file_name"] = df_states["file_name"].apply(lambda x: x[:2])
tem_df = df_states[["file_name", "state", "agency"]].drop_duplicates().groupby(["file_name", "state"]).count()
tem_df = tem_df.join(
                pd.value_counts(df_states["state"]),
                on = "state"
            ).join(
                smm_df,
                on = "file_name"
            )
tem_df.columns = ['count_agency', 'count_csv', 'sum', 'min', 'max']
tem_df = tem_df.sort_values(['count_agency', 'count_csv', 'sum', 'min', 'max'], kind= 'heapsort')
print(tem_df[tem_df.count_agency == 1])

                             count_agency  count_csv  sum  min  max
file_name state                                                    
HP        Himachal Pradesh              1          1   23   23   23
JK        Jammu and Kashmir             1          1   23   23   23
MZ        Mizoram                       1          1   23   23   23
NL        Nagaland                      1          1   23   23   23
SK        Sikkim                        1          1   24   24   24
AR        Arunachal Pradesh             1          1   25   25   25
PY        Puducherry                    1          1   25   25   25
JH        Jharkhand                     1          2   29   10   19
TR        Tripura                       1          2   47   23   24
MN        Manipur                       1          2   48   24   24
ML        Meghalaya                     1          2   49   24   25
UK        Uttarakhand                   1          3   66   16   25
CH        Chandigarh                    1       

Looking at the output I chose Andhra Pradesh state. The reason is that this state have 10 .csv files within 23 to 25 column with data.
Rajasthan state could work too but my laptop wouldn't have been able to manage.

In [81]:
try:
    df_states.drop(columns= ["agency", "station_location", "start_month"], inplace= True)
except:
    pass
print(df_states.head(2))

  file_name           state        city  start_month_num  start_year
0        AP  Andhra Pradesh    Tirupati                7        2016
1        AP  Andhra Pradesh  Vijayawada                5        2017


In [82]:
df_states[df_states['state'] == 'Andhra Pradesh'].city.unique()

array(['Tirupati', 'Vijayawada', 'Visakhapatnam', 'Rajamahendravaram',
       'Amaravati', 'Anantapur', 'Chittoor', 'Kadapa'], dtype=object)

We are working with a large dataset which is split in multiple files. 
First I will create a function that will return a dataframe combining all datasheets measurements in a given state, so will see what data I'm working with.

In [83]:
#def combine_file_state()